<a href="https://colab.research.google.com/github/Praful22/CasadiPlayground/blob/main/CasADi_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CasADi Notebook: Example and Tutorials Developed to learn and work for Coordinated Control of Aerial Vehicle.

CasADi is a software tool for numerical optimization focusing on optimal control problems.

*   Creators: Joel Andersson, Joris Gillis, Moritz Diehl



This document is a colloborative notebook in the form of bullet points, tutorials, and shareable-knowledge that Pratik Gyawali and Praful Sigdel noted during their work for Aerial Vehicle Coordinated Control.

After going throught this notebook, one should be able to formulate and manipulate expressions in CasADi’s symbolic framework, generate derivative information efficiently using algorithmic differentiation, to set up, solve and perform forward and adjoint sensitivity analysis for systems of ordinary differential equations (ODE) or differential-algebraic equations (DAE) as well as to formulate and solve nonlinear programs (NLP) problems and optimal control problems (OCP).[1]
---
1.   Programming Languages Availibility:
*   Python (Best Documented) --> This is what we have chosen to work with.
*   C++ (Limited Documentation)
*   Matlab/Octave

Par Performance among all three programming languages.

2. What is CasADi and What it is not? [1]
* Tool for Algorithmic Differentiation using syntax borrowed from Computer Algebra Systems (CAS).
* Core functionalities of the Tool:
  * Algorithmic Differentiation
  * Support for Ordinary Differential Equations (ODE) / Differential-Algebraic Equations(DAE) integration
  * Sensitivity Analysis
  * Nonlinear programming
  * Interfaces to other numerical tools.

3. Usable only through the implementation or reimplementation of models in CasADi syntax.

4. CasADi provides the user with a set of “building blocks” that can be used to implement general- purpose or specific-purpose OCP solvers efficiently with a modest programming effort. Not an "optimal control problem solver" that allows the user to enter an OCP and then gives the solution back.

5. Installation Instructions: http://install.casadi.org/

# Symbolic Framework


*   Self-contained symbolic framework that allows users to create symbolic experessions using everything-is-a-matrix syntax i.e. vectors are treated as n-by-1 matrices and scalars as 1-by-1 matrix.
*   All matrices are sparse and use a **general sparse format - [compressed column storage (CCS)](https://en.wikipedia.org/wiki/Sparse_matrix#:~:text=Compressed%20sparse%20column%20(CSC%20or%20CCS))** - to store matrices.



Install Casadi!

In [2]:
pip install casadi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.5/67.5 MB 13.3 MB/s eta 0:00:00


The SX data type is used to represent matrices whose elements consist of symbolic expressions made up by a sequence of unary and binary operations. One can import the symbols into the workspace as follows:

In [3]:
from casadi import *

Create a variable *x* using below syntax :

In [4]:
x = MX.sym("x")
x

MX(x)

This creates a 1-by-1 matrix, i.e. a scalar containing a symbolic primitive called x. This is just the display name, not the identifier. Multiple variables can have the same name, but still be different. The identifier is the return value. You can also create vector- or matrix-valued symbolic variables by supplying additional arguments to SX.sym:

In [12]:
x = SX.sym('y',3)
Z = SX.sym('Z',5,2)
print(x)
print(Z)

[y_0, y_1, y_2]

[[Z_0, Z_5], 
 [Z_1, Z_6], 
 [Z_2, Z_7], 
 [Z_3, Z_8], 
 [Z_4, Z_9]]


which creates a 3-by-1 matrix, i.e. a vector, and a 5-by-2 matrix with symbolic primitives, respectively.

SX.sym is a (static) function which returns an SX instance. When variables have been declared, expressions can now be formed in an intuitive way:

In [13]:
x = SX.sym('x') # Declaration
f = x**3 + 10 # Expression
f = sqrt(f)
print(f)

sqrt(((x*sq(x))+10))


Create constant SX instances without any symbolics primitives as follows:

In [14]:
A1 = SX.zeros(3,3)
A2 = SX(4,5)
A3 = SX.eye(4)
print(A1, A2, A3)

@1=0, 
[[@1, @1, @1], 
 [@1, @1, @1], 
 [@1, @1, @1]] 
[[00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00]] @1=1, 
[[@1, 00, 00, 00], 
 [00, @1, 00, 00], 
 [00, 00, @1, 00], 
 [00, 00, 00, @1]]


The following list summarizes the most commonly used ways of constructing new SX expressions:
*  SX.sym(name,n,m): Create an 𝑛-by-𝑚 symbolic primitive
*  SX.zeros(n,m): Create an 𝑛-by-𝑚 dense matrix with all zeros
*  SX(n,m): Create an 𝑛-by-𝑚 sparse matrix with all structural zeros
*  SX.ones(n,m): Create an 𝑛-by-𝑚 dense matrix with all ones
* SX.eye(n): Create an 𝑛-by-𝑛 diagonal matrix with ones on the diagonal and structural zeros elsewhere.
* SX(scalar_type): Create a scalar (1-by-1 matrix) with value given by the argument. This method can be used explicitly, e.g. SX(9), or implicitly, e.g. 9 * SX.ones(2,2).
* SX(matrix_type): Create a matrix given a numerical matrix given as a NumPy or SciPy matrix (in Python) or as a dense or sparse matrix (in MATLAB/Octave). In MATLAB/Octave e.g. SX([1,2,3,4]) for a row vector, SX([1;2;3;4]) for a column vector and SX([1,2;3,4]) for a 2-by-2 matrix. This method can be used explicitly or implicitly.
* repmat(v,n,m): Repeat expression 𝑣 𝑛 times vertically and 𝑚 times horizontally. repmat(SX(3),2,1) will create a 2-by-1 matrix with all elements 3.
* (Python only) SX(list): Create a column vector (𝑛-by-1 matrix) with the elements in the list, e.g. SX([1,2,3, 4]) (note the difference between Python lists and MATLAB/Octave horizontal concatenation, which both uses square bracket syntax)
*(Python only) SX(list of list): Create a dense matrix with the elements in the lists, e.g. SX([[1,2],[3, 4]]) or a row vector (1-by-𝑛 matrix) using SX([[1,2,3,4]]).

MX Symbolics:

*
*
*
*
*

